In [6]:

import pandas as pd
import os
import pandas as pd
from datetime import datetime

def find_relevant_files(folder_path, text1):
    relevant_files = []
    for file_name in os.listdir(folder_path):
        if text1 in file_name:
            relevant_files.append(file_name)
    return relevant_files


folder_path = r'C:\work_folder\RFB2\Catheter_testing\20250905_f28p_height'
# read summary csv files
text1= '.csv'
text2= '_octiv'
text3= 'rfb'
csvfiles= find_relevant_files(folder_path, text1)


In [ ]:
def parse_file_name(file_name):
    SN = file_name[0:5]
    # target = "Brush" if "BRUSH" in file_name.upper() else "F28P"
    target = "Brush"
    if '1p' in file_name:
        Ht = 2.50
    elif '2p' in file_name:
        Ht = 2.56
    elif '3p' in file_name:
        Ht = 2.62
    else:
        Ht = 2.43  # default height
    #if "brush" in file_name:
    #    target= "Brush"
    #elif "F28P" in file_name:
    #    target= "F28P"
    return SN, target, Ht

testfile = '24560rF28P.csv'
[SN, target, ht] = parse_file_name(testfile)
print(parse_file_name(testfile))



('24560', 'Brush', 2.43)


In [8]:
def get_file_creation_date(file_path):
    creation_time = os.path.getctime(file_path)
    creation_date = datetime.fromtimestamp(creation_time).strftime('%Y-%m-%d %H:%M:%S')
    return creation_date

fp= r'C:\work_folder\RFB2\Catheter_testing\20250905_f28p_height'
dt= get_file_creation_date(fp)
print(dt)

2025-09-05 15:13:55


In [ ]:
def get_peak_pow(fpath):
    
    # Open the file and read line 14 (index 13)
    with open(fpath, 'r') as file:
        lines = file.readlines()
        line_14 = lines[13]  # Line 14 is at index 13

    # Split the line by comma to get columns
    columns = line_14.strip().split(',')

    # Get the 2nd column (index 1)
    aa = columns[1]

    print("Value in 2nd column of line 14:", aa)
    return aa


def read_and_store_data(folder_path, text1):
    all_data = []
    relevant_files = find_relevant_files(folder_path, text1)
    
    for file_name in relevant_files:
        print(file_name)
        file_path = os.path.join(folder_path, file_name)

        # Read Peak Power
        peak_pow= get_peak_pow(file_path)

        # Get SN, target, and date of file creation from file name
        SN, target, Ht = parse_file_name(file_name)
        creation_date = get_file_creation_date(file_path)
        
        # Add all these to a dataframe
        df = pd.DataFrame({
            'SN': [SN],
            'target': [target],
            'DoF': [creation_date],
            'Ht': [Ht],
            'peak_pow': [peak_pow]
        })

        
        all_data.append(df)
    
    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

folder_path = r'C:\work_folder\RFB2\Catheter_testing\20250905_f28p_height'
# read summary csv files
text1= '.csv'
text2= '_octiv'
text3= 'rfb'
summary_df= read_and_store_data(folder_path, text1)

18383brush.csv
Value in 2nd column of line 14:  3.310000e+01
18383brush2.csv
Value in 2nd column of line 14:  3.280000e+01
18383brush3.csv
Value in 2nd column of line 14:  3.220000e+01
18383f28p.csv
Value in 2nd column of line 14:  3.390000e+01
18383f28p2.csv
Value in 2nd column of line 14:  3.310000e+01
output.csv


IndexError: list index out of range

In [5]:
writepath= folder_path + '\output.csv'
summary_df.to_csv(writepath)

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
C:\Users\Santosh.Behera\AppData\Local\Temp\ipykernel_33528\4031658145.py:1: SyntaxWarning: invalid escape sequence '\o'
  writepath= folder_path + '\output.csv'


In [ ]:
summary_df["sample_repeat_no"].unique()

In [ ]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def create_gui(data):
    def plot_sensor_data(sensor, parse_by):
        fig, ax = plt.subplots()
        if parse_by == 'test':
            for test in data['test_no'].unique():
                test_data = data[data['test_no'] == test]
                ax.scatter( test_data['test_no'], test_data[sensor], label=f'Test {test}')
            ax.boxplot([data[data['test_no'] == test][sensor] for test in data['test_no'].unique()], positions=range(len(data['test_no'].unique())))
        elif parse_by == 'operator':
            for operator in data['operator_no'].unique():
                operator_data = data[data['operator_no'] == operator]
                ax.scatter(operator_data['operator_no'], operator_data[sensor], label=f'Operator {operator}')
            ax.boxplot([data[data['operator_no'] == operator][sensor] for operator in data['operator_no'].unique()], positions=range(len(data['operator_no'].unique())))
        
        ax.set_xlabel(parse_by)
        ax.set_ylabel(sensor)
        ax.legend()
        canvas = FigureCanvasTkAgg(fig, master=window)
        canvas.draw()
        canvas.get_tk_widget().pack()

    window = tk.Tk()
    window.title("Sensor Data Plotter")

    sensor_label = tk.Label(window, text="Select Attribute to plot:")
    sensor_label.pack()
    sensor_var = tk.StringVar()
    sensor_dropdown = ttk.Combobox(window, textvariable=sensor_var)
    sensor_dropdown['values'] = ['Ewatts', 'Awatts', 'Eff', 'PWatts', 'G3Eff', 'Real', 'Img', 'Freq']
    sensor_dropdown.pack()

    parse_by_label = tk.Label(window, text="Parse By:")
    parse_by_label.pack()
    parse_by_var = tk.StringVar()
    parse_by_dropdown = ttk.Combobox(window, textvariable=parse_by_var)
    parse_by_dropdown['values'] = ['test', 'operator']
    parse_by_dropdown.pack()

    plot_button = tk.Button(window, text="Plot", command=lambda: plot_sensor_data(sensor_var.get(), parse_by_var.get()))
    plot_button.pack()

    window.mainloop()

# Example usage
create_gui(summary_df)



In [ ]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np

def create_gui2(data):
    def plot_sensor_data(sensor, parse_by, color_by):
        fig, ax = plt.subplots()
        unique_values = data[parse_by].unique()
        colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_values)))
        
        for i, value in enumerate(unique_values):
            subset = data[data[parse_by] == value]
            scatter = ax.scatter(subset[parse_by], subset[sensor], label=f'{parse_by} {value}', color=colors[i])
            ax.boxplot([subset[subset[parse_by] == value][sensor] for value in unique_values], positions=range(len(unique_values)))
        
        if color_by:
            scatter.set_array(data[color_by].astype('category').cat.codes)
            plt.colorbar(scatter, ax=ax, label=color_by)
        
        ax.set_ylabel(sensor)
        ax.set_xlabel(parse_by)
        ax.legend()
        canvas = FigureCanvasTkAgg(fig, master=window)
        canvas.draw()
        canvas.get_tk_widget().pack()

    window = tk.Tk()
    window.title("Sensor Data Plotter")

    sensor_label = tk.Label(window, text="Select Sensor:")
    sensor_label.pack()
    sensor_var = tk.StringVar()
    sensor_dropdown = ttk.Combobox(window, textvariable=sensor_var)
    sensor_dropdown['values'] = ['Ewatts', 'Awatts', 'Eff', 'PWatts', 'G3Eff', 'Real', 'Img', 'Freq']
    sensor_dropdown.pack()

    parse_by_label = tk.Label(window, text="Parse By:")
    parse_by_label.pack()
    parse_by_var = tk.StringVar()
    parse_by_dropdown = ttk.Combobox(window, textvariable=parse_by_var)
    parse_by_dropdown['values'] = ['test_no', 'operator_no', 'target_type', 'sample_repeat_no']
    parse_by_dropdown.pack()

    color_by_label = tk.Label(window, text="Color By:")
    color_by_label.pack()
    color_by_var = tk.StringVar()
    color_by_dropdown = ttk.Combobox(window, textvariable=color_by_var)
    color_by_dropdown['values'] = ['SN', 'target_type', 'operator_no', 'test_no']
    color_by_dropdown.pack()

    plot_button = tk.Button(window, text="Plot", command=lambda: plot_sensor_data(sensor_var.get(), parse_by_var.get(), color_by_var.get()))
    plot_button.pack()

    window.mainloop()




In [ ]:
summary_df= summary_df[summary_df["Eff"]>60]
create_gui2(summary_df)

In [ ]:
summary_df["SN"].unique()

In [ ]:
summary_df.head

In [ ]:
summary_df.to_csv("writepath")

In [ ]:
df1= pd.read_csv('writepath')
df1.head

In [ ]:
writepath

In [ ]:
writepath= r'C:\work_folder\Misc_codes\prelim_gage\Data\output.csv'
summary_df.to_csv(writepath)

In [ ]:
writepath

In [ ]:
df1= pd.read_csv('writepath')